In [24]:
# 스크래핑에 필요한 라이브러리를 불러와봅시다.

from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [25]:
from selenium.common.exceptions import NoSuchElementException
driver = webdriver.Chrome(service=Service("C:/DataEngineering/chromedriver/chromedriver-win64/chromedriver.exe"))
driver.get("https://ticket.melon.com/")
#driver.implicitly_wait(5)


In [26]:
wait = WebDriverWait(driver, 10)  # 최대 10초까지 기다림

try:
    checkbox = wait.until(EC.element_to_be_clickable((By.ID, "popup_chk")))
    #checkbox = driver.find_element(By.ID, "popup_chk")
    ActionChains(driver).click(checkbox).perform()

    closed = wait.until(EC.element_to_be_clickable((By.ID, "btn_layerpopup_close")))
    #closed = driver.find_element(By.ID, "btn_layerpopup_close")
    ActionChains(driver).click(closed).perform()
    
except NoSuchElementException:
    print("체크박스 또는 닫기 버튼이 없음 - 팝업 건너뜀")


# 이건 무조건 실행
#concert_banner = driver.find_element(By.CLASS_NAME, "btn_gnb_menu.btn_gnb.btn_g_menu02")
#ActionChains(driver).click(concert_banner).perform()

try:
    concert_banner = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "btn_gnb_menu.btn_gnb.btn_g_menu02")))
    ActionChains(driver).click(concert_banner).perform()
except TimeoutException:
    print("콘서트 배너를 찾지 못했습니다.")

In [19]:
#print(driver.page_source)

In [31]:
from datetime import datetime
import traceback
import requests
driver.implicitly_wait(100)
li_elements = driver.find_elements(By.CSS_SELECTOR, "#perf_poster > li")
concerts=[]

for index, li in enumerate(li_elements):
    try:
        concert = {}

        # a 태그 및 href 추출
        try:
            a_tag = li.find_element(By.TAG_NAME, "a")
            href = a_tag.get_attribute("href")
            # 콘텐츠가 비어있으면 href값 자체가 # 이었음 
            # "#" in href 로 조건문 걸어버리면 먼가 자꾸 반복문이 중단되어서(아마도 개발자가 링크뒤에 주석을 넣었거나 그랬을수있음#) href.strip()== "#"으로 비교
            if  href.strip() == "#":
                print(f"{index + 1}번째 공연에서 href='#' 발견됨. 크롤링 중단.")
                break
        except Exception as e:
            print(f"{index + 1}번째 공연: <a> 태그 추출 실패 - {e}")
            continue

        # title 추출
        try:
            #멜론에서 한거라
            concert["site"]="멜론티켓"
            title = li.find_element(By.CSS_SELECTOR, 'strong.tit').text.strip()
            concert["title"] = title
            #print(title)
        except Exception as e:
            print(f"{index + 1}번째 공연: 제목 추출 실패 - {e}")
            concert["title"] = "정보 없음"
            continue

        # 날짜 추출 및 변환
        try:
            day_text = li.find_element(By.CSS_SELECTOR, 'span.day').text.strip()
            if '-' in day_text:
                start_date, end_date = [d.strip() for d in day_text.split('-')]
            else:
                start_date = end_date = day_text

            concert["start_date"] = datetime.strptime(start_date, "%Y.%m.%d").isoformat()
            concert["end_date"] = datetime.strptime(end_date, "%Y.%m.%d").isoformat()
        except Exception as e:
            print(f"{index + 1}번째 공연: 날짜 추출 또는 변환 실패 - {e}")
            concert["start_date"] = concert["end_date"] = None
            continue

        # 장소 추출
        try:
            location = li.find_element(By.CSS_SELECTOR, 'span.location').text.strip()
            concert["location"] = location
            

        except Exception as e:
            print(f"{index + 1}번째 공연: 장소 추출 실패 - {e}")
            concert["location"] = "정보 없음"
            continue

        concerts.append(concert)
        #print(f"{index + 1}번째 공연 크롤링 완료: {concert}")
        #break

    except Exception as e:
        print(f"{index + 1}번째 공연: 예기치 못한 오류 발생")
        traceback.print_exc()
        #break
        continue

print("크롤링 완료된 전체 공연 목록:")
print(concerts)
print(len(concerts))
# driver.quit()



크롤링 완료된 전체 공연 목록:
[{'site': '멜론티켓', 'title': '전주국제영화제 씨네투어X음악', 'start_date': '2025-05-02T00:00:00', 'end_date': '2025-05-06T00:00:00', 'location': '전주시네마타운'}, {'site': '멜론티켓', 'title': '제17회 서울재즈페스티벌 2025', 'start_date': '2025-05-30T00:00:00', 'end_date': '2025-06-01T00:00:00', 'location': '올림픽공원'}, {'site': '멜론티켓', 'title': 'AoB Presents #1 : Grateful Sound', 'start_date': '2025-05-13T00:00:00', 'end_date': '2025-05-13T00:00:00', 'location': 'KT&G 상상마당 홍대 라이브홀'}, {'site': '멜론티켓', 'title': 'LUCY 일곱 번째 단독 콘서트 〈와장창〉', 'start_date': '2025-05-02T00:00:00', 'end_date': '2025-05-04T00:00:00', 'location': '고려대학교 화정체육관'}, {'site': '멜론티켓', 'title': 'KARDI 단독콘서트 ‘각별’ : 롤링 30주년 기념 공연', 'start_date': '2025-05-17T00:00:00', 'end_date': '2025-05-17T00:00:00', 'location': '롤링홀'}, {'site': '멜론티켓', 'title': '윤종신 전국투어 콘서트 ‘그때’ - 서울', 'start_date': '2025-06-07T00:00:00', 'end_date': '2025-06-08T00:00:00', 'location': '우리금융아트홀'}, {'site': '멜론티켓', 'title': 'JEONJINHEE PIANO 雨後', 'start_date': '2025-05-10T

In [ ]:
#test 해본거
from datetime import datetime
import traceback
import requests
s=datetime.now()
e=datetime.now()
concerts={'title':'전주국제영화제 씨네투어X음악', 'location':'전주시네마타운','start_date':s.isoformat(),'end_date':e.isoformat()}
requests.post("http://127.0.0.1:8000/allincon_api/crawling/", json=concerts)

<Response [201]>

In [ ]:
#서버로 정보 전송
import requests 
#print(concerts)
response = requests.post("http://127.0.0.1:8000/api/concerts/", json=concerts)
print(response)


<Response [201]>


In [35]:
print(concerts)

[{'site': '멜론티켓', 'title': '전주국제영화제 씨네투어X음악', 'start_date': '2025-05-02T00:00:00', 'end_date': '2025-05-06T00:00:00', 'location': '전주시네마타운'}, {'site': '멜론티켓', 'title': '제17회 서울재즈페스티벌 2025', 'start_date': '2025-05-30T00:00:00', 'end_date': '2025-06-01T00:00:00', 'location': '올림픽공원'}, {'site': '멜론티켓', 'title': 'AoB Presents #1 : Grateful Sound', 'start_date': '2025-05-13T00:00:00', 'end_date': '2025-05-13T00:00:00', 'location': 'KT&G 상상마당 홍대 라이브홀'}, {'site': '멜론티켓', 'title': 'LUCY 일곱 번째 단독 콘서트 〈와장창〉', 'start_date': '2025-05-02T00:00:00', 'end_date': '2025-05-04T00:00:00', 'location': '고려대학교 화정체육관'}, {'site': '멜론티켓', 'title': 'KARDI 단독콘서트 ‘각별’ : 롤링 30주년 기념 공연', 'start_date': '2025-05-17T00:00:00', 'end_date': '2025-05-17T00:00:00', 'location': '롤링홀'}, {'site': '멜론티켓', 'title': '윤종신 전국투어 콘서트 ‘그때’ - 서울', 'start_date': '2025-06-07T00:00:00', 'end_date': '2025-06-08T00:00:00', 'location': '우리금융아트홀'}, {'site': '멜론티켓', 'title': 'JEONJINHEE PIANO 雨後', 'start_date': '2025-05-10T00:00:00', 'end_da